In [2]:
import pandas as pd
import numpy as np

## Arrythmia

In [206]:
arrythmia = pd.read_csv('arrhythmia.data', header = None)
arrythmia[1] = arrythmia[1].astype(np.bool)
arrythmia[279] = ~(arrythmia[279] == 1)
arrythmia = (arrythmia.rename({1: 'gender'}, axis = 1)
             .replace({'?': None})
             .astype('float64')
             .astype({'gender':'bool', 279: 'bool'})
             .apply(lambda x: x.fillna(x.mean())))
arrythmia.to_csv('../data/arrythmia.csv', index = False)

,0,gender,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,False,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,True
1,56,True,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,True
2,54,False,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,True
3,55,False,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,False
4,75,False,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,True


## Compas

In [5]:
race_cols = ['African_American','Hispanic','Asian','Hispanic','Native_American','Other']

compas = (pd.read_csv('compass.csv')
 .assign(Caucasian = lambda df: df[race_cols].sum(axis=1) == 0,
        Y= lambda df: df.Two_yr_Recidivism)
 .drop('Two_yr_Recidivism', axis = 1)
 .query('Caucasian or African_American')
 .drop(race_cols, axis = 1)
 .rename({'Female':'gender', 'Caucasian': 'race'}, axis=1)
 .astype({'Y': 'bool'}))
#compas.to_csv('../data/compas.csv', index=False)

In [6]:
compas

,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,gender,Misdemeanor,race,Y
1,0,0,0,0,0,0,False,True
2,4,0,0,1,0,0,False,True
4,14,1,0,0,0,0,True,True
6,0,0,0,0,1,1,True,False
7,0,0,0,0,0,0,True,False
...,...,...,...,...,...,...,...,...
6165,0,0,0,0,0,1,False,True
6166,0,1,0,1,0,0,False,False
6167,0,1,0,1,0,0,False,False
6168,0,0,0,1,0,0,False,False


In [7]:
compas.shape

(5278, 8)

## German

In [283]:
german = (pd.read_csv('german.data', header = None, delimiter = ' ')
          .replace(['A91','A93','A94'],True)
          .replace(['A92','A95'], False)
          .rename({8:'gender'},axis=1)
          .assign(foreign = lambda df: df[19] == 'A201',
                 Y = lambda df: df[20]-1)
          .drop([19,20], axis=1)
          .astype({'Y':'bool'})
          .to_csv('../data/german.csv', index = False))

## Drug

In [286]:
drug = pd.read_csv('drug_consumption.data', header = None)
drug['race'] = drug[5] == -0.31685
drug['Y'] = drug.loc[:,29] == 'CL0'
#drug.loc[:,31-18::]
drug = drug.drop(np.arange(31-18,32), axis = 1)
drug = drug.drop([0,5], axis = 1)
drug.to_csv('../data/drug.csv', index = False)

Adult

In [10]:
adult =pd.read_csv('adult.csv')
adult.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X11,X12,X13,X14,gender,Y
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,2174,0,40,United-States,True,False
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,0,0,13,United-States,True,False
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,0,0,40,United-States,True,False
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,0,0,40,United-States,True,False
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,40,Cuba,False,False


## Default

In [3]:
default = (pd.read_excel('default of credit card clients.xls')
           .drop(0)
           .drop('Unnamed: 0', axis=1)
           .astype('float64')
           .assign(X2= lambda df: df.X2 == 1)
           .astype({'Y': 'bool', 
                   'X2':'bool'}))


#default.to_csv('../data/default.csv', index = False)

In [4]:
default.shape

(30000, 24)

## CART Benchmarking

In [337]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = drug.drop('Y', axis = 1).to_numpy()
Y = drug['Y'].to_numpy()
g = drug['race'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.7729342564448948  with fairness 0.0
Best Fairness:  0.7729342564448948  with fairness 0.0


In [327]:
arrythmia.head()

,0,gender,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75.0,False,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,True
1,56.0,True,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,True
2,54.0,False,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,True
3,55.0,False,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,False
4,75.0,False,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,True


In [338]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = arrythmia.drop(279, axis = 1).to_numpy()
Y = arrythmia[279].to_numpy()
g = arrythmia['gender'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.6745893719806764  with fairness 0.2562670662670663
Best Fairness:  0.6745893719806764  with fairness 0.2562670662670663


In [4]:
def getFold(X,Y,group, indices):
    X_train = np.delete(X, indices, axis=0)
    Y_train = np.delete(Y, indices)
    g_train = np.delete(group, indices)

    X_test = X[indices,:]
    Y_test = Y[indices]
    g_test = group[indices]
    
    return X_train, Y_train, g_train, X_test, Y_test, g_test

def fairCrossValScore(model, X, Y, g, cv=10, hp=10):
    break_points = np.floor(np.arange(0,1+1/cv, 1/cv) * X.shape[0]).astype(np.int)
    acc = []
    eqGap = []
    for i in range(cv):
        X_tr, Y_tr, g_tr, X_t, Y_t, g_t = getFold(X, Y, g, np.arange(break_points[i], break_points[i+1]))
        model.fit(X_tr, Y_tr)
        preds = model.predict(X_t)
        accuracy = (preds == Y_t).mean()
        EqOfOpGap = abs((preds[g_t & Y_t] == 1).mean() - (preds[(~g_t) & Y_t] == 1).mean())
        acc.append(accuracy)
        eqGap.append(EqOfOpGap)
    return np.array(acc), np.array(eqGap)


In [308]:
cv=10
break_points = np.floor(np.arange(0,1+1/cv, 1/cv) * X.shape[0]).astype(np.int)
break_points

array([   0,  188,  377,  565,  754,  942, 1131, 1319, 1508, 1696, 1885])

In [299]:
cross_val_score?

In [292]:
min_samples[np.argmax(results)]

200

In [91]:
adult = pd.get_dummies(pd.read_csv('adult.csv'))

adult[~adult.gender].Y.value_counts(True)

False    0.890539
True     0.109461
Name: Y, dtype: float64

In [99]:
adult['Y'].value_counts(True)

False    0.75919
True     0.24081
Name: Y, dtype: float64

In [100]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score
adult = pd.get_dummies(pd.read_csv('adult.csv'))

X = adult.drop('Y', axis = 1).to_numpy()
Y = adult['Y'].to_numpy()
g = adult['gender'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc],'(', acc_sd[best_acc],')', 
      ' with fairness', fair[best_acc], '(', fair_sd[best_acc], ')')
print('Best Fairness: ', acc[best_fair],'(', acc_sd[best_fair],')', 
      ' with fairness', fair[best_fair], '(', fair_sd[best_fair], ')')


Best accuracy:  0.8551027403460625 ( 0.003079335862558836 )  with fairness 0.15898151111784337 ( 0.05118031600416503 )
Best Fairness:  0.853751473862005 ( 0.00580938136509807 )  with fairness 0.08377761643140332 ( 0.043578983809011845 )


In [59]:
adult['Y'].value_counts(True)

False    0.75919
True     0.24081
Name: Y, dtype: float64

In [61]:
adult_bench_cart = (pd.DataFrame({'algo': 'cart', 'Accuracy': acc+[0.76], 'EqOfOp': fair+[0]})
                      .sort_values('EqOfOp')
                     .assign(cumAcc = lambda df: df.Accuracy.cummax()))
adult_bench_cart

,algo,Accuracy,EqOfOp,cumAcc
11,cart,0.760000,0.000000,0.760000
0,cart,0.853751,0.083778,0.853751
7,cart,0.839317,0.126175,0.853751
2,cart,0.854611,0.155069,0.854611
1,cart,0.855103,0.158982,0.855103
5,cart,0.845183,0.169443,0.855103
6,cart,0.841989,0.169890,0.855103
3,cart,0.853199,0.172683,0.855103
10,cart,0.824361,0.175668,0.855103
4,cart,0.850895,0.180885,0.855103


In [101]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import cross_val_score

adult = pd.get_dummies(pd.read_csv('adult.csv'))

X = adult.drop('Y', axis = 1).to_numpy()
Y = adult['Y'].to_numpy()
g = adult['gender'].to_numpy()

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []

for C in Cs:
    cart = LogisticRegression(C = C, max_iter = 1000)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc],'(', acc_sd[best_acc],')', 
      ' with fairness', fair[best_acc], '(', fair_sd[best_acc], ')')
print('Best Fairness: ', acc[best_fair],'(', acc_sd[best_fair],')', 
      ' with fairness', fair[best_fair], '(', fair_sd[best_fair], ')')


Best accuracy:  0.8009889774358611 ( 0.011605607007162344 )  with fairness 0.07064164785058671 ( 0.07957451472730119 )
Best Fairness:  0.7979177243646081 ( 0.005852524789344624 )  with fairness 0.035770340121590964 ( 0.03234977513381819 )


In [63]:
adult_bench_lr = (pd.DataFrame({'algo': 'logreg', 'Accuracy': acc+[0.76], 'EqOfOp': fair+[0]})
                      .sort_values('EqOfOp')
                     .assign(cumAcc = lambda df: df.Accuracy.cummax()))

In [64]:
adult_bench_cart.append(adult_bench_lr).to_csv('benchmark_adult.csv')

In [28]:
adult_bench = pd.DataFrame({'algo': 'logReg', 'Accuracy': acc, 'EqOfOp': fair})
bestOfAct = np.argmax(adult_bench.Accuracy)
bestOfFair = np.argmin(adult_bench.EqOfOp)
adult_bench.iloc[[bestOfAct, bestOfFair],:]

,algo,Accuracy,EqOfOp
4,logReg,0.800989,0.070642
2,logReg,0.797918,0.035770


In [22]:
adult_full_bench = adult_bench.append(adult_bench_cart)
(adult_full_bench.groupby(['algo'])
 .sort_values('Accuracy', ascending = True)
 .assign(cumAcc=lambda df: df.Accuracy.cummax()))

AttributeError: 'DataFrameGroupBy' object has no attribute 'assign'

In [415]:
default = (pd.read_excel('default of credit card clients.xls')
           .drop(0)
           .drop('Unnamed: 0', axis=1)
           .astype('float64')
           .assign(X2= lambda df: df.X2 == 1)
           .astype({'Y': 'bool', 
                   'X2':'bool'}))


default.to_csv('../data/default.csv', index = False)

In [102]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score
default = pd.get_dummies(pd.read_csv('../data/default.csv'))

X = default.drop('Y', axis = 1).to_numpy()
Y = default['Y'].to_numpy()
g = default['X2'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc],'(', acc_sd[best_acc],')', 
      ' with fairness', fair[best_acc], '(', fair_sd[best_acc], ')')
print('Best Fairness: ', acc[best_fair],'(', acc_sd[best_fair],')', 
      ' with fairness', fair[best_fair], '(', fair_sd[best_fair], ')')

Best accuracy:  0.8205666666666668 ( 0.014791927227752008 )  with fairness 0.02990852760752783 ( 0.02736945382781393 )
Best Fairness:  0.8196 ( 0.014105790773059606 )  with fairness 0.025732411356625656 ( 0.018744686631095474 )


In [82]:
default_bench_cart = (pd.DataFrame({'algo': 'cart', 'Accuracy': acc, 'EqOfOp': fair})
                      .sort_values('EqOfOp')
                     .assign(cumAcc = lambda df: df.Accuracy.cummax()))
default_bench_cart

,algo,Accuracy,EqOfOp,cumAcc
10,cart,0.819600,0.025732,0.819600
0,cart,0.812400,0.026186,0.819600
9,cart,0.819133,0.026249,0.819600
5,cart,0.819800,0.028369,0.819800
6,cart,0.819800,0.028369,0.819800
7,cart,0.819800,0.028369,0.819800
8,cart,0.820567,0.029909,0.820567
1,cart,0.818167,0.037382,0.820567
4,cart,0.819667,0.042691,0.820567
2,cart,0.818967,0.048100,0.820567


In [103]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import cross_val_score

default = pd.get_dummies(pd.read_csv('../data/default.csv'))

X = default.drop('Y', axis = 1).to_numpy()
Y = default['Y'].to_numpy()
g = default['X2'].to_numpy()

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []

for C in Cs:
    cart = LogisticRegression(C = C, max_iter = 1000)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc],'(', acc_sd[best_acc],')', 
      ' with fairness', fair[best_acc], '(', fair_sd[best_acc], ')')
print('Best Fairness: ', acc[best_fair],'(', acc_sd[best_fair],')', 
      ' with fairness', fair[best_fair], '(', fair_sd[best_fair], ')')

Best accuracy:  0.7787666666666667 ( 0.017237588127242305 )  with fairness 0.0 ( 0.0 )
Best Fairness:  0.7787666666666667 ( 0.017237588127242305 )  with fairness 0.0 ( 0.0 )


In [84]:
default_bench_lr = (pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 'EqOfOp': fair})
                      .sort_values('EqOfOp')
                     .assign(cumAcc = lambda df: df.Accuracy.cummax()))
default_bench_lr.append(default_bench_cart).to_csv('benchmark_default.csv')

In [86]:
default['Y'].value_counts(True)

False    0.7788
True     0.2212
Name: Y, dtype: float64

In [ ]:
default_bench_lr = pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 'EqOfOp': fair})
bestOfAct = np.argmax(default_bench_lr.Accuracy)
bestOfFair = np.argmin(default_bench_lr.EqOfOp)
default_bench_lr.iloc[[bestOfAct, bestOfFair],:]

In [419]:
min_samples[np.argmax(acc)]

600

In [360]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = default.drop('Y', axis = 1).to_numpy()
Y = default['Y'].to_numpy()
g = default['X2'].to_numpy()
arrythmia = pd.read_csv('arrhythmia.data', header = None)
arrythmia[1] = arrythmia[1].astype(np.bool)
arrythmia[279] = ~(arrythmia[279] == 1)
arrythmia = (arrythmia.rename({1: 'gender'}, axis = 1)
             .replace({'?': None})
             .astype('float64')
             .astype({'gender':'bool', 279: 'bool'})
             .apply(lambda x: x.fillna(x.mean())))
arrythmia.to_csv('../data/arrythmia.csv', index = False)
min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmax(acc)
print('Best accuracy: ', acc[best_acc], ' with fairness', fair[best_acc])
print('Best Fairness: ', acc[best_fair], ' with fairness', fair[best_fair])

Best accuracy:  0.8205666666666668  with fairness 0.02990852760752783
Best Fairness:  0.8205666666666668  with fairness 0.02990852760752783


In [66]:
compas['Y'].value_counts(True)

False    0.529557
True     0.470443
Name: Y, dtype: float64

In [104]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import cross_val_score

X = compas.drop('Y', axis = 1).to_numpy()
Y = compas['Y'].to_numpy()
g = compas['race'].to_numpy()

min_samples = [50,100,150,200,250,400,450,500,600,700,1000]
acc = []
acc_sd = []
fair = []
fair_sd = []

for sam in min_samples:
    cart = tree.DecisionTreeClassifier(min_samples_leaf = sam)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc],'(', acc_sd[best_acc],')', 
      ' with fairness', fair[best_acc], '(', fair_sd[best_acc], ')')
print('Best Fairness: ', acc[best_fair],'(', acc_sd[best_fair],')', 
      ' with fairness', fair[best_fair], '(', fair_sd[best_fair], ')')

Best accuracy:  0.6813211574952562 ( 0.01866495508086602 )  with fairness 0.2556830606880317 ( 0.06189231213216089 )
Best Fairness:  0.6582089874072796 ( 0.022512105923808215 )  with fairness 0.21339935693287387 ( 0.06110967988966843 )


In [78]:
compas_bench_cart = (pd.DataFrame({'algo': 'cart', 'Accuracy': acc, 'EqOfOp': fair})
                      .sort_values('EqOfOp')
                     .assign(cumAcc = lambda df: df.Accuracy.cummax()))
compas_bench_cart

,algo,Accuracy,EqOfOp,cumAcc
10,cart,0.658209,0.213399,0.658209
7,cart,0.659348,0.213696,0.659348
8,cart,0.659348,0.213696,0.659348
9,cart,0.659348,0.213696,0.659348
3,cart,0.666542,0.244025,0.666542
4,cart,0.666733,0.246561,0.666733
5,cart,0.668249,0.247001,0.668249
6,cart,0.668249,0.247001,0.668249
2,cart,0.664836,0.248195,0.668249
0,cart,0.681321,0.255683,0.681321


In [105]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import cross_val_score

compas = pd.read_csv('../data/compas.csv')
X = compas.drop('Y', axis = 1).to_numpy()
Y = compas['Y'].to_numpy()
g = compas['race'].to_numpy()

Cs = [0.01, 0.1, 1, 3, 10]
acc = []
acc_sd = []
fair = []
fair_sd = []

for C in Cs:
    cart = LogisticRegression(C = C)
    scores = fairCrossValScore(cart, X, Y, g, cv=10)
    acc.append(scores[0].mean())
    acc_sd.append(scores[0].std())
    fair.append(scores[1].mean())
    fair_sd.append(scores[1].std())


best_acc = np.argmax(acc)
best_fair = np.argmin(fair)
print('Best accuracy: ', acc[best_acc],'(', acc_sd[best_acc],')', 
      ' with fairness', fair[best_acc], '(', fair_sd[best_acc], ')')
print('Best Fairness: ', acc[best_fair],'(', acc_sd[best_fair],')', 
      ' with fairness', fair[best_fair], '(', fair_sd[best_fair], ')')

Best accuracy:  0.6809466821919383 ( 0.016144503877351193 )  with fairness 0.27129156909169494 ( 0.07576579543021579 )
Best Fairness:  0.6809466821919383 ( 0.016144503877351193 )  with fairness 0.27129156909169494 ( 0.07576579543021579 )


In [80]:
compas_bench_lr = (pd.DataFrame({'algo': 'logreg', 'Accuracy': acc, 'EqOfOp': fair})
                      .sort_values('EqOfOp')
                     .assign(cumAcc = lambda df: df.Accuracy.cummax()))
compas_bench_lr.append(compas_bench_cart).to_csv('benchmark_compas.csv')